In [3]:
## INPUT
name="Rogers5thEd"

In [15]:
import pandas as pd
import re
from bs4 import BeautifulSoup

skills={'ACROBATICS',
 'ANIMAL HANDLING',
 'ARCANA',
 'ATHLETICS',
 'DECEPTION',
 'HISTORY',
 'INSIGHT',
 'INTIMIDATION',
 'INVESTIGATION',
 'MEDICINE',
 'NATURE',
 'PERCEPTION',
 'PERFORMANCE',
 'PERSUASION',
 'RELIGION',
 'SLEIGHT OF HAND',
 'STEALTH',
 'SURVIVAL'}


rawtext = open('raw/'+name+'.html', 'r').read()
soup = BeautifulSoup(rawtext, 'html.parser')
msgs=soup.findAll('div',
             {'class': lambda x: x 
                       and 'message' in x.split()
             }
            )

In [16]:
plyr=None
when=None
df = pd.DataFrame(columns=['Player','Timestamp','Skill','Result', 'Raw','Plus'])
for c,m in enumerate(msgs):
    print(c+1,"/",len(msgs), end='\r')
    
    who=m.findAll('span',{'class': 'by'})
    if who!=[]:
        plyr=who[0].renderContents()
        when=m.find('span',{'class': 'tstamp'}).renderContents()
    
    log=True
    skchk=m.findAll('div',{'class': ['sheet-rolltemplate-skill','sheet-rolltemplate-simple']})
    if skchk!=[]:   
        skrolls=skchk[0].find_all('div',{'class': ['sheet-roll','sheet-adv','sheet-solo']})
        if len(skrolls)==1:
            roll=skrolls[0]
        elif len(skrolls)==2:
            roll=skrolls[0] if skrolls[1].find('span',{'class': 'sheet-grey'})!=None else skrolls[1]
        else:
            print("err: number of rolls is off")
            print(c)
            log=False
        result=roll.find('span',{'class': lambda x: x and 'inlinerollresult' in x}).renderContents()
    
        tmpskill=skchk[0].find('div',{'class': 'sheet-label'}).renderContents()
        tmpskill=re.sub(r'[^ A-Z0123456789-]','',str(tmpskill))
        tmpskill=tmpskill.split(' ')
        skill= [" ".join(tmpskill[:-1]),tmpskill[-1]] if len(tmpskill)>2 else tmpskill
        
        
        if (skill[0]!='') and log and (skill[0] in skills):
            df.at[c, 'Player'] = re.sub(':','',plyr.decode('UTF-8'))
            df.at[c, 'Timestamp'] = when.decode('UTF-8')
            df.at[c, 'Skill'] = str(skill[0])
            df.at[c, 'Result'] = result.decode('UTF-8')
            df.at[c, 'Raw'] = str(int(result)-int(skill[1]))
            df.at[c, 'Plus'] = str(skill[1])
df.to_csv("skills_"+name+".csv",index=False,sep=";")
df

err: number of rolls is off
598
err: number of rolls is off
620
err: number of rolls is off5943 594359435943
2999
err: number of rolls is off
3974
err: number of rolls is off
4781
err: number of rolls is off
5445
err: number of rolls is off
5719


,Player,Timestamp,Skill,Result,Raw,Plus
206,Nura,"March 21, 2020 7:22PM",INSIGHT,7,2,5
316,Lui,"March 26, 2020 8:52PM",INSIGHT,20,15,5
343,Zoanne,"March 26, 2020 9:26PM",PERCEPTION,12,13,-1
347,Korth,"March 26, 2020 9:37PM",PERCEPTION,25,20,5
348,Nura,"March 26, 2020 9:38PM",PERCEPTION,14,9,5
...,...,...,...,...,...,...
5834,Who is Rea,"January 07, 2021 11:09PM",PERCEPTION,12,11,1
5872,Kääpiöinën,8:46PM,SURVIVAL,8,7,1
5883,Korth,8:54PM,ATHLETICS,20,16,4
5900,Korth,9:08PM,ATHLETICS,22,18,4
